In [94]:
import tensorflow as tf
import tensorflow_hub as hub

import glob
import numpy as np
import cv2

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [29]:
IMG_WIDTH = image_size[0]
IMG_HEIGHT = image_size[1]

classes = ["daisy", "dandelion", "roses", "sunflowers", "tulips"]
n_classes = len(classes)
data_folder = "./flower_photos/"

In [91]:
tf.reset_default_graph()

# Load a pre-trained TF-Hub module for extracting features from images. We've
# chosen this particular module for speed, but many other choices are available.
image_module = hub.Module('https://tfhub.dev/google/inaturalist/inception_v3/feature_vector/1')
image_size = hub.get_expected_image_size(image_module)

# define tf graph
X = tf.placeholder(dtype=tf.float32, shape=(None,IMG_WIDTH,IMG_HEIGHT,3), name='X')
y = tf.placeholder(dtype=tf.int32, shape=(None,),name='y')

features = image_module(X)
#hidden1 = tf.layers.dense(inputs=features, units=100, name='hidden1', activation='relu')
logits = tf.layers.dense(inputs=features, units=n_classes, name='logits', activation=None)

with tf.name_scope('loss'):
    xentropy = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y),name='xentropy')

with tf.name_scope('optimizer'):
    optimizer = tf.train.GradientDescentOptimizer(0.01)
    training_op = optimizer.minimize(xentropy)
    
with tf.name_scope('accuracy'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [35]:
def preprecessing(img):
    resized = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    return resized

In [64]:
# load data set
data = []
target = []

for i, cls in enumerate(classes):
    path = data_folder+cls+"/*.jpg"
    image_paths = glob.glob(path)
    for img_path in image_paths:
        img = cv2.imread(img_path)
        processed_img = preprecessing(img)
        data.append(processed_img)
        target.append(i)

data = np.array(data)
target = np.array(target)

In [ ]:
# feature scaling
scaled_data = StandardScaler().fit_transform(data)

In [39]:
data.shape

(3670, 299, 299, 3)

In [67]:
# split validation set
X_train, X_val, y_train, y_val = train_test_split(data, target,test_size=0.3,random_state=4)

In [77]:
batch_size = 100
n_batches = int(np.ceil(X_train.shape[0]/batch_size))
n_epochs = 10


In [84]:
def fetch_batch (X, y, batch_index, batch_size):
    # load data
    start_index = batch_index*batch_size
    if (start_index + batch_size > X.shape[0]):
        end_index = X.shape[0] 
    else:    
        end_index = start_index + batch_size 
    
    X_curr = X[start_index : end_index,:]

    y_curr = y[start_index : end_index]
    
    return X_curr, y_curr

In [93]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for ep in range(n_epochs):
        for i in range(n_batches):
            X_batch, y_batch = fetch_batch(X_train, y_train, i, batch_size)
            sess.run(training_op, feed_dict={X:X_batch, y:y_batch})
            
        loss_train, acc_train = sess.run([xentropy, accuracy], feed_dict={X:X_batch, y:y_batch})
        #acc_val = accuracy.eval(feed_dict={X:X_val, y:y_val})
        print("--------------------------------")
        print("Loss: ", loss_train)
        print("Accuracy train: ", acc_train)
        #print("Accuracy test: ", acc_val)

--------------------------------
Loss:  73.38793
Accuracy train:  0.44444445
--------------------------------
Loss:  50.547134
Accuracy train:  0.7777778


KeyboardInterrupt: 